In [1]:
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers_interpret import SequenceClassificationExplainer

# Instanciando modelo

In [2]:
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"
CHECKPOINT_PATH = "../../data/VICTOR/checkpoints/checkpoint-149136"

In [3]:
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(CHECKPOINT_PATH)

# Carregando exemplos para predição

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [5]:
TRAIN_PATH = "/home/kenzosaki/mestrado/data/VICTOR/train_small.csv"
TEST_PATH = "/home/kenzosaki/mestrado/data/VICTOR/test_small.csv"

In [6]:
train_df = pd.read_csv(TRAIN_PATH, index_col=0)
test_df = pd.read_csv(TEST_PATH, index_col=0)

In [9]:
label_encoder = LabelEncoder()
label_encoder.fit(train_df["document_type"])

LabelEncoder()

In [10]:
label_encoder.classes_

array(['acordao_de_2_instancia', 'agravo_em_recurso_extraordinario',
       'despacho_de_admissibilidade', 'outros', 'peticao_do_RE',
       'sentenca'], dtype=object)

# Analisando corpus

In [42]:
def print_tokenization(text: str) -> None:
    print(f"-- {text}")
    print("--{}".format(" ".join(tokenizer.convert_ids_to_tokens(tokenizer.encode(text, add_special_tokens=False, max_length=512, truncation=True)))))

In [21]:
test_df.head()

,themes,process_id,file_name,document_type,pages,body
0,[33],AI_850120,AI_850120_789456_93_11052013.pdf,outros,1,processo termo remessa tendo vista disposto po...
1,[33],AI_850127,AI_850127_273628856_1280_28102014.pdf,outros,1,supremo tribunal federal ofício brasília de ou...
2,[232],AI_855935,AI_855935_1366318_50_29062013.pdf,outros,1,excelentíssimo senhor juiz presidente turma re...
3,[232],AI_855935,AI_855935_1366318_50_29062013.pdf,outros,2,advogados agravado vitor rodrigues moura oab c...
4,[232],AI_855935,AI_855935_1366318_50_29062013.pdf,outros,3,eminentes ministros tempestividade publicação ...


In [43]:
SAMPLE = 3
for text in test_df.sample(SAMPLE)["body"].values:
    print_tokenization(text)
    print("--" * 10)

-- atividade inclusive quando decorrentes transformação reclassificação cargo função que deu aposentadoria que serviu referência para concessão pensão forma lei ARTIGO_2º aplica aos proventos aposentadorias dos servidores públicos que aposentarem forma caput ARTIGO_6º emenda constitucional disposto ARTIGO_7º mesma emenda ARTIGO_3º ressalvado direito opção aposentadoria pelas normas estabelecidas pelo ARTIGO_40 constituição federal pelas regras estabelecidas pelos ARTIGO_2º emenda constitucional servidor união dos estados distrito federal dos municípios incluídas suas autarquias fundações que tenha ingressado serviço público até de dezembro poderá aposentar com proventos integrais desde que preencha cumulativamente seguintes condições parágrafo único aplica ao valor dos proventos aposentadorias concedidas com base neste artigo disposto ARTIGO_7º emenda constitucional observando igual critério revisão pensões derivadas dos proventos servidores falecidos que tenham aposentado conformidade

# Realizando inferências

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer,
    custom_labels=label_encoder.classes_
)

In [13]:
FILTER_BY = "acordao_de_2_instancia"

In [14]:
sampled_df = test_df[test_df["document_type"] == FILTER_BY]
example = sampled_df.sample(1)
ex_text = example["body"].values[0]
ex_label = example["document_type"].values[0]
ex_text, ex_label

('isentando todavia respectivo pagamento face ARTIGO_3o LEI_1060 deferindo justiça gratuita mas ressalvado disposto nos seus ARTIGO_11o como voto jorge andré carvalho mendonça juiz federal relator acórdão vistos etc decide segunda turma recursal dos juizados especiais federais pernambuco unanimidade negar provimento recurso nos termos voto acima recife data julgamento jorge andré carvalho mendonça juiz federal relator',
 'acordao_de_2_instancia')

In [15]:
word_attributions = cls_explainer(ex_text, class_name=ex_label)

In [16]:
_ = cls_explainer.visualize()